In [1]:
!git clone https://github.com/k223kim/CryptocurrencyPredictionRNN.git

Cloning into 'CryptocurrencyPredictionRNN'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 16 (delta 2), reused 16 (delta 2), pack-reused 0
Unpacking objects: 100% (16/16), done.


In [2]:
pwd

'/content'

In [0]:
import pandas as pd
import os
from sklearn import preprocessing #to scale
from collections import deque
import random
import numpy as np

In [0]:
# df = pd.read_csv("/content/CryptocurrencyPredictionRNN/crypto_data/LTC-USD.csv", names=["time", "low", "high", "open", "close", "volume"])

In [0]:
# print(df.head())

         time        low       high       open      close      volume
0  1528968660  96.580002  96.589996  96.589996  96.580002    9.647200
1  1528968720  96.449997  96.669998  96.589996  96.660004  314.387024
2  1528968780  96.470001  96.570000  96.570000  96.570000   77.129799
3  1528968840  96.449997  96.570000  96.570000  96.500000    7.216067
4  1528968900  96.279999  96.540001  96.500000  96.389999  524.539978


In [0]:
SEQ_LEN = 60 #update once a minute
FUTURE_PERIOD_PREDICT = 3 #how many period forward? 1period = 1 min
RATIO_TO_PREDICT = "LTC-USD" #what are we predicting?

In [0]:
#create the target (since it is a supervised learning)

def classify(current, future):
  if float(future) > float(current):
    return 1 #you should buy this!
  else:
    return 0

In [0]:
def preprocess_df(df):
  #scaling
  df = df.drop('future', 1) #don't need future column (target column)
  
  for col in df.columns:
    if col != "target":
      df[col] = df[col].pct_change() #this normalize the data
      df.dropna(inplace=True)
      df[col] = preprocessing.scale(df[col].values) #from sklearn, scale from 0 to 1
  
  
  df.dropna(inplace=True) #just in case (if it creates something that is not a number, we will drop!)
  
  sequential_data = [] #list
  prev_days = deque(maxlen = SEQ_LEN) 
  #we keep append to the list and as it reaches the length of 60, it pops out the old item
  
  #print to double check
#   print(df.head())
  
#   for c in df.columns:
#     print(c)
  
  #iterate over the columns
  for i in df.values: 
    #df.values; converts the dataframe into a list of lists
    #would not contain time
    #still in the order of the index
    #it will contain target
    
    #i is the row of all the columns (close, volume, etc)
    
    prev_days.append([n for n in i[:-1]]) 
    #append a list
    #n for n : each value in that list of list (each of the columns)
    #up to the last i; so we are not taking the target
    
    if len(prev_days) == SEQ_LEN:
      sequential_data.append([np.array(prev_days), i[-1]])
      #append the features and the labels in a numpy arrays
      #sequence of 60 feature sets
      #current label: i[-1] (target; 0 or a 1)
     
   
  random.shuffle(sequential_data)
  
  #balance data
  buys = []
  sells = []
  
  for seq, target in sequential_data:
    if target == 0:
      sells.append([seq, target])
    elif target == 1:
      buys.append([seq, target])
      
  random.shuffle(buys)
  random.shuffle(sells)
  
  #what is the minimum value?
  lower = min(len(buys), len(sells))
  
  buys = buys[:lower]
  sells = sells[:lower]
  
  sequential_data = buys+sells
  random.shuffle(sequential_data)
  
  #split in x and y
  
  X = []
  y = []
  
  for seq, target in sequential_data:
    X.append(seq)
    y.append(target)
    
  return np.array(X), y

In [0]:
#visualize (making sure to comment later)
# preprocess_df(main_df)
#analysis:
#everything is converted to percent change and are scaled,
#the columns are in order

#the sequences will be a sequence of price and volume and NOT a target (cheating!)


In [32]:
main_df = pd.DataFrame()

ratios = ["BTC-USD", "LTC-USD", "ETH-USD", "BCH-USD"]
for ratio in ratios:
  dataset = f"/content/CryptocurrencyPredictionRNN/crypto_data/{ratio}.csv"
  
  df = pd.read_csv(dataset, names=["time", "low", "high", "open", "close", "volume"])
  print(df.head())
  
  #merge all dataframes
  #for all dataframes their columns are named the same
  #so we need a new column which distinguishes between those dataframes
  #inplace means that we don't have to redefine the dataframes
  df.rename(columns={"close": f"{ratio}_close", "volume":f"{ratio}_volume"}, inplace = True)

  df.set_index("time", inplace=True)
  df = df[[f"{ratio}_close", f"{ratio}_volume"]]

  if len(main_df) == 0: # if it is empty
    main_df = df
  else:
    main_df = main_df.join(df)
    
#get future price
#future price is based on the close column
#shift the close column up by how many periods forward you want to predict


# print(main_df.head())
# for c in main_df.columns:
#   print(c) #double check the columns


main_df["future"] = main_df[f"{RATIO_TO_PREDICT}_close"].shift(-FUTURE_PERIOD_PREDICT)

print(main_df[[f"{RATIO_TO_PREDICT}_close", "future"]].head())
#e.g. current price: 96.58 predicted future price: 96.5


         time          low         high         open        close    volume
0  1528968660  6489.549805  6489.560059  6489.560059  6489.549805  0.587100
1  1528968720  6487.370117  6489.560059  6489.549805  6487.379883  7.706374
2  1528968780  6479.410156  6487.370117  6487.370117  6479.410156  3.088252
3  1528968840  6479.410156  6479.419922  6479.419922  6479.410156  1.404100
4  1528968900  6475.930176  6479.979980  6479.410156  6479.979980  0.753000
         time        low       high       open      close      volume
0  1528968660  96.580002  96.589996  96.589996  96.580002    9.647200
1  1528968720  96.449997  96.669998  96.589996  96.660004  314.387024
2  1528968780  96.470001  96.570000  96.570000  96.570000   77.129799
3  1528968840  96.449997  96.570000  96.570000  96.500000    7.216067
4  1528968900  96.279999  96.540001  96.500000  96.389999  524.539978
         time        low   high        open      close     volume
0  1528968720  485.98999  486.5  486.019989  486.01001  26

In [33]:
#map the function to the new column "target"
#converts the output to a list which we will assign as a column
main_df["target"]= list(map(classify, main_df[f"{RATIO_TO_PREDICT}_close"], main_df["future"]))

print(main_df[[f"{RATIO_TO_PREDICT}_close", "future", "target"]].head(10))
#confirm if the target works (0-bad, 1-good)

            LTC-USD_close     future  target
time                                        
1528968660      96.580002  96.500000       0
1528968720      96.660004  96.389999       0
1528968780      96.570000  96.519997       0
1528968840      96.500000  96.440002       0
1528968900      96.389999  96.470001       1
1528968960      96.519997  96.400002       0
1528969020      96.440002  96.400002       0
1528969080      96.470001  96.400002       0
1528969140      96.400002  96.400002       0
1528969200      96.400002  96.400002       0


In [36]:
#make sequence, balance, normalize, scale
#sequential data; cannot just shuffle and take a random percentage
#the model is more likely to over-fit the in-sample data and out-sample data
#take a chunk of the sequences and seperate them away
#for time-series data: out-sample data must be a chunck of data in the future
#in the case of our data: take the last 5 percent of the historical data and seperate that out
#i.e. as if we built the model 5% of the time ago and then forward test it

#true out of sample?
#out of sample testing?
#true forward test?

times = sorted(main_df.index.values) #.values => converts to numpy array
last_5pct = times[-int(0.05*len(times))]
print(last_5pct)

#separate the validation data and out of sample data and training data

validation_main_df = main_df[(main_df.index >= last_5pct)]
#training data
main_df = main_df[(main_df.index < last_5pct)]

train_x, train_y = preprocess_df(main_df)
validation_x, validation_y = preprocess_df(validation_main_df)

print(f"train data: {len(train_x)} validation: {len(validation_x)}")
print(f"Don't buys: {train_y.count(0)}, buys: {train_y.count(1)}")
print(f"VALIDATION Don't buys: {validation_y.count(0)}, buys: {validation_y.count(1)}")

1534643640
train data: 65962 validation: 3174
Don't buys: 32981, buys: 32981
VALIDATION Don't buys: 1587, buys: 1587
